In [1]:
import pandas as pd

In [2]:
from fastcore.test import *
import operator

In [3]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
print(df)

   A  B
0  1  4
1  2  5
2  3  6


In [4]:
test([1, 2], [1, 2], operator.eq)

In [5]:
def _fail():
    raise Exception()


test_fail(_fail)

In [6]:
test([1, 2], [1, 2], operator.eq)
test_fail(lambda: test([1, 2], [1], operator.eq))

In [7]:
test(["abc"], ["abc"], all_equal)
test_fail(lambda: test(["abc"], ["cab"], all_equal))

NameError: name 'all_equal' is not defined

In [8]:
def _fail_args(a):
    print(a)
    if a == 5:
        raise ValueError


test_fail(_fail_args, args=(5,))
test_fail(_fail_args, kwargs=dict(a=5))

5
5


In [9]:
_fail_args(5)

5


ValueError: 

In [10]:
ifnone(a, b)

NameError: name 'ifnone' is not defined

In [11]:
test_eq(ifnone(None, 1), 1)
test_eq(ifnone(2, 1), 2)

NameError: name 'ifnone' is not defined